In [1]:
import sys
sys.path.append("../processing/")

In [2]:
import gzip
import json
import numpy as np
from scipy import stats as sp_stats

from _config import Config
from utils import write_dicts_to_csv

In [3]:
datasets = ["news", "midterms"]

fields_to_exclude = [
    "features",
    "features_excluded"
]

conf = Config()
out_fpath = f"{conf.modeling_dir}/prefix/runs_csvs/res_q50_nested.csv"

In [4]:
all_runs = []

for dataset in datasets:
    # paths
    fpath = f"{conf.modeling_dir}/prefix/runs/{dataset}_q50_nested_cv.json.gz"
    
    # load
    runs = json.load(gzip.open(fpath))

    # aggregate metrics and filter fields
    runs_agg = []

    for run in runs:
        run_agg = {}

        for f_name, f_val in run.items():
            
            if f_name in fields_to_exclude:
                continue
                
            elif f_name == "clf":
                run_agg["clf_name"] = f_val
            
            elif f_name == "estimator":
                # if run["clf"] == "GB":
                #     print(
                #         run["clf"], 
                #         run["dataset"],
                #         run["prefix"],
                #         run["feature_groups"],
                #         run["estimator"]
                #     )
                continue
                
            elif isinstance(f_val, list):
                run_agg[f"{f_name}__mean"] = np.mean(f_val)
                run_agg[f"{f_name}__sem"] = sp_stats.sem(f_val)

            else:
                run_agg[f_name] = f_val
    
        runs_agg.append(run_agg)

    all_runs += runs_agg
    
print("len(all_runs) =", len(all_runs))

len(all_runs) = 440


In [5]:
write_dicts_to_csv(all_runs, out_fpath)